In [24]:
import pandas as pd
import numpy as np

In [25]:
df = pd.read_csv("data/red_wine_quality.csv")

In [26]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [27]:
#Removing duplicates
df.drop_duplicates(inplace=True)

In [28]:
#Separating Dependent & Independent Data

X = df.drop(labels = ["quality"], axis = 1)
Y = df[["quality"]]

In [29]:
#Defiinig the numerical cols for preprocessing
num_cols = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"]

In [30]:
#Importing libs for simple imputer for missing values & Standard scaler for preprocessing

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [31]:
#For pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

In [32]:
#Numerical Pipeline
num_pipeline = Pipeline(
    steps = [
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
         
    ]
)

In [33]:
preprocessor = ColumnTransformer([
    ("num_pipeline", num_pipeline, num_cols)
])

In [34]:
#Training Dataset
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state=100)

In [36]:
#Transforming the data

X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

In [38]:
X_train.head()

,num_pipeline__fixed acidity,num_pipeline__volatile acidity,num_pipeline__citric acid,num_pipeline__residual sugar,num_pipeline__chlorides,num_pipeline__free sulfur dioxide,num_pipeline__total sulfur dioxide,num_pipeline__density,num_pipeline__pH,num_pipeline__sulphates,num_pipeline__alcohol
0,-0.814164,0.831393,-1.389797,-0.011751,0.276543,0.551176,-0.311659,-0.050228,0.987656,-0.508962,0.162165
1,-0.238627,-1.572634,0.624678,-0.702978,-0.616997,-0.501659,-0.667325,0.003400,0.087919,1.124410,-1.613689
2,-0.065966,-1.194473,0.624678,-0.242160,-0.580526,-0.980221,-1.111907,-0.715224,0.023652,0.089941,0.722961
3,-0.583949,2.046913,-0.886178,-0.549372,-0.288758,0.359751,0.251478,0.014126,0.280720,-0.563407,-1.146359
4,-0.756610,0.372197,-0.785455,-0.242160,-0.124638,-0.310235,-0.578409,-0.050228,1.373258,-0.236733,-0.211699


In [40]:
## Model Training 

from sklearn.linear_model import LogisticRegression, RidgeClassifier, LogisticRegressionCV, LassoCV, ElasticNetCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [41]:
#Model Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [44]:
def evaluate_model(true, pred):
    mae = mean_absolute_error(true, pred)
    mse = mean_squared_error(true, pred)
    rmse = np.sqrt(mean_squared_error(true, pred))
    r2_squr = r2_score(true, pred)
    
    return mae, mse, rmse, r2_squr

In [43]:
#Model Trainer 

models = {
    "LogisticRegression" : LogisticRegression(),
    "RidgeClassifier" : RidgeClassifier(),
    "LogsiticregressionCV" : LogisticRegressionCV(),
    "LassoCV" : LassoCV(),
    "ElasticNetCV" : ElasticNetCV(),
    "DecisionTree" : DecisionTreeClassifier(),
    "SVC" : SVC(),
    "RandomForest" : RandomForestClassifier(),
    "GradientClassifier" : GradientBoostingClassifier()
}

In [45]:
model_list = []
r2_list = []

In [46]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    mae, mse, rmse, r2_square = evaluate_model(y_test, y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print("Model Training Performance")
    print("RMSE", rmse)
    print("MSE", mae)
    print("R2 score", r2_square*100)
    
    r2_list.append(r2_square)
    
    print("*"*100)

e:\ML_Projects\red_wine_quality\venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\ML_Projects\red_wine_quality\venv\lib\site-packages\sklearn\linear_model\_ridge.py:1182: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\ML_Projects\red_wine_quality\venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\ML_Projects\red_wine_quality\venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL 

LogisticRegression
Model Training Performance
RMSE 0.773329952663334
MSE 0.47058823529411764
R2 score 18.484855928664402
****************************************************************************************************
RidgeClassifier
Model Training Performance
RMSE 0.7749130248475834
MSE 0.49754901960784315
R2 score 18.15077746935565
****************************************************************************************************


e:\ML_Projects\red_wine_quality\venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\ML_Projects\red_wine_quality\venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

LogsiticregressionCV
Model Training Performance
RMSE 0.7717436331412898
MSE 0.4730392156862745
R2 score 18.81893438797315
****************************************************************************************************
LassoCV
Model Training Performance
RMSE 0.6886876297186275
MSE 0.5258985198600051
R2 score 35.35228165551061
****************************************************************************************************
ElasticNetCV
Model Training Performance
RMSE 0.6887622606265393
MSE 0.5260735842159656
R2 score 35.33826955735173
****************************************************************************************************
DecisionTree
Model Training Performance
RMSE 0.8574929257125442
MSE 0.553921568627451
R2 score -0.2235377926257387
****************************************************************************************************
SVC
Model Training Performance
RMSE 0.7524469885568253
MSE 0.4485294117647059
R2 score 22.827875899678183
******************************

e:\ML_Projects\red_wine_quality\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\ML_Projects\red_wine_quality\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\ML_Projects\red_wine_quality\venv\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ayush Gandhi\AppData\Local\Temp\ipykernel_13144\2916354809.py:3: DataConversionWarning: A column-vector y was passed when a 1

RandomForest
Model Training Performance
RMSE 0.765365477223457
MSE 0.4485294117647059
R2 score 20.155248225208165
****************************************************************************************************


e:\ML_Projects\red_wine_quality\venv\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientClassifier
Model Training Performance
RMSE 0.7717436331412898
MSE 0.4681372549019608
R2 score 18.81893438797315
****************************************************************************************************
